# ACOPF with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl) using [PowerModels.jl](https://github.com/lanl-ansi/PowerModels.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports non-linear AC optimal power flow through a deep integration
with [PowerModels.jl](https://github.com/lanl-ansi/PowerModels.jl). This example shows a
single multi-period optimization of economic dispatch with a full representation of
AC optimal power flow.

## Dependencies
We can use the same RTS data and some of the initialization as in
[OperationsProblem example](../../notebook/PowerSimulations_examples/1_operations_problems.ipynb)
by sourcing it as a dependency.

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "PowerSimulations_examples", "1_operations_problems.jl"));

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/s

Since we'll be doing non-linear optimization, we need a solver that supports non-linear
problems. Ipopt is quite good.

In [2]:
using Ipopt
solver = optimizer_with_attributes(Ipopt.Optimizer)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[])

In the [OperationsProblem example](../../notebook/PowerSimulations_examples/1_operations_problems.ipynb)
we defined a unit-commitment problem with a copper plate representation of the network.
Here, we want do define an economic dispatch (linear generation decisions) with an ACOPF
network representation.
So, starting with the network, we can select from _almost_ any of the endpoints on this
tree:

In [3]:
TypeTree(PSI.PM.AbstractPowerModel,  init_expand = 10, scopesep="\n")

PowerModels
AbstractPowerModel
├──PowerModels
│  AbstractACRModel
│  ├──PowerModels
│  │  ACRPowerModel
│  └──PowerModels
│     AbstractIVRModel
│     └──PowerModels
│        IVRPowerModel (0 children)
├──PowerModels
│  AbstractACPModel
│  └──PowerModels
│     ACPPowerModel
├──PowerModels
│  AbstractConicModel
│  ├──PowerModels
│  │  AbstractWRConicModel
│  │  └──PowerModels
│  │     AbstractSOCWRConicModel (1 children)
│  └──PowerModels
│     AbstractWRMModel
│     └──PowerModels
│        AbstractSDPWRMModel (2 children)
├──PowerModels
│  AbstractWRModel
│  ├──PowerModels
│  │  AbstractQCWRModel
│  │  ├──PowerModels
│  │  │  AbstractQCRMPowerModel (1 children)
│  │  └──PowerModels
│  │     AbstractQCLSModel (1 children)
│  └──PowerModels
│     AbstractSOCWRModel
│     └──PowerModels
│        SOCWRPowerModel (0 children)
├──PowerModels
│  AbstractLPACModel
│  └──PowerModels
│     AbstractLPACCModel
│     └──PowerModels
│        LPACCPowerModel (0 children)
├──PowerModels
│  AbstractBFModel
│  ├──PowerModels
│  │  AbstractBFAModel
│  │  └──PowerModels
│  │     BFAPowerModel (0 children)
│  ├──PowerModels
│  │  AbstractBFConicModel
│  │  └──PowerModels
│  │     AbstractSOCBFConicModel (1 children)
│  └──PowerModels
│     AbstractBFQPModel
│     └──PowerModels
│        AbstractSOCBFModel (1 children)
├──PowerModels
│  AbstractActivePowerModel
│  ├──PowerSimulations
│  │  AreaBalancePowerModel
│  ├──PowerSimulations
│  │  CopperPlatePowerModel
│  └──PowerModels
│     AbstractDCPModel
│     ├──PowerModels
│     │  AbstractDCPLLModel (1 children)
│     ├──PowerModels
│     │  AbstractNFAModel (1 children)
│     ├──PowerModels
│     │  DCPPowerModel (0 children)
│     ├──PowerModels
│     │  AbstractDCMPPModel (1 children)
│     └──PowerSimulations
│        StandardPTDFModel (0 children)
└──PowerModels
   AbstractACTModel
   └──PowerModels
      ACTPowerModel

For now, let's just choose a standard ACOPF formulation.

In [4]:
ed_template = template_economic_dispatch(network = ACPPowerModel)


Operations Problem Specification

  transmission:  PowerModels.ACPPowerModel
  devices: 
      ILoads:
        device_type = PowerSystems.InterruptibleLoad
        formulation = PowerSimulations.InterruptiblePowerLoad
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.FixedOutput
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalRampLimited
      DistRE:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.FixedOutput
      Hydro:
        device_type = PowerSystems.HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchReservoirFlow
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RE:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
  branches: 
      T:
        device_type = PowerSyst

for some reason the HydroROR is currently incompatible with ACOPF. Bug report has been filed.

In [5]:
delete!(ed_template.devices, :HydroROR)

Dict{Symbol,PowerSimulations.DeviceModel} with 6 entries:
  :ILoads     => DeviceModel{InterruptibleLoad,InterruptiblePowerLoad}(Interrup…
  :Generators => DeviceModel{ThermalStandard,ThermalRampLimited}(ThermalStandar…
  :DistRE     => DeviceModel{RenewableFix,FixedOutput}(RenewableFix, FixedOutpu…
  :Hydro      => DeviceModel{HydroEnergyReservoir,HydroDispatchReservoirFlow}(H…
  :Loads      => DeviceModel{PowerLoad,StaticPowerLoad}(PowerLoad, StaticPowerL…
  :RE         => DeviceModel{RenewableDispatch,RenewableFullDispatch}(Renewable…

Now we can build a 4-hour economic dispatch / ACOPF problem with the RTS data.

In [6]:
problem = OperationsProblem(
    EconomicDispatchProblem,
    ed_template,
    sys,
    horizon = 4,
    optimizer = solver,
    balance_slack_variables = true,
)

┌ Info: Unit System changed to SYSTEM_BASE
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/base.jl:353
┌ Warning: The data doesn't include devices of type PowerSystems.InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: Data doesn't contain generators with ramp limits, consider adjusting your formulation
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/devices/thermal_generation.jl:644
┌ Warning: The data doesn't include devices of type PowerSystems.Transformer2W, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/devices_models/device_constructors/common/constructor_validations.jl:3



Operations Problem Specification

  transmission:  PowerModels.ACPPowerModel
  devices: 
      ILoads:
        device_type = PowerSystems.InterruptibleLoad
        formulation = PowerSimulations.InterruptiblePowerLoad
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalRampLimited
      DistRE:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.FixedOutput
      Hydro:
        device_type = PowerSystems.HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchReservoirFlow
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RE:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
  branches: 
      T:
        device_type = PowerSystems.Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
        device_type = PowerSyst

And solve it ...

In [7]:
solve!(problem)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    15152
Number of nonzeros in inequality constraint Jacobian.:     5572
Number of nonzeros in Lagrangian Hessian.............:    21120

Total number of variables............................:     5832
                     variables with only lower bounds:     1168
                variables with lower and upper bounds:     3292
                     variables with only upper bounds:        0
Total number of equa

Results
========

Variables
=========

P__ThermalStandard: (4, 76)
P__RenewableDispatch: (4, 30)
γ⁺__Q: (4, 73)
FqFT__TapTransformer: (4, 15)
FpTF__Line: (4, 105)
Q__HydroEnergyReservoir: (4, 19)
Q__RenewableDispatch: (4, 30)
P__HydroEnergyReservoir: (4, 19)
γ⁺__P: (4, 73)
FqTF__HVDCLine: (4, 1)
FpFT__Line: (4, 105)
FpFT__HVDCLine: (4, 1)
Q__ThermalStandard: (4, 76)
FqTF__Line: (4, 105)
FqTF__TapTransformer: (4, 15)
Vm__Bus: (4, 73)
FqFT__Line: (4, 105)
theta__Bus: (4, 73)
FpTF__TapTransformer: (4, 15)
γ⁻__P: (4, 73)
FpFT__TapTransformer: (4, 15)
FqFT__HVDCLine: (4, 1)
FpTF__HVDCLine: (4, 1)
γ⁻__Q: (4, 73)


Optimizer Log
-------------
        timed_solve_time = 24.37056311
        solve_bytes_alloc = 4756689617
        solve_time = 9.919086933135986
        obj_value = 1.973167740235758e7
        solver = Ipopt
        sec_in_gc = 1.249752329
        dual_status = FEASIBLE_POINT
        primal_status = FEASIBLE_POINT
        termination_status = LOCALLY_SOLVED


Total Cost: OBJECTIVE_FUNCTION = 1.973167740235758e7

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*